# **주식 데이터 - 뉴스기사 키워드 추출을 통한 주식 종목 이슈 분석**

In [ ]:
#한글 폰트 다운로드
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 17 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (24.7 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120901 files and direc

In [ ]:
#크롤링 위한 user_agent 다운로드
!pip install user_agent

  Preparing metadata (setup.py) ... done
  Created wheel for user_agent: filename=user_agent-0.1.10-py3-none-any.whl size=18966 sha256=19d718ca6c8acdac7c253a27f38d7e89128cb97a6b8537f0c4b714be6a38f0ad
  Stored in directory: /root/.cache/pip/wheels/69/29/26/1956a891a058037774285ee79ab5c3ecf034dba50a4198fedd
Successfully built user_agent


In [ ]:
#한글 형태소 분석기 다운로드
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 37.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 4.1 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1231 sha256=98f26a8505320e5441fe6ef104d553f762467610ace6c2acc8abfdaedec493b7
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  1743k      0 --:--:-- --:--:-- --:--:-- 1743k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/ex

In [ ]:
#주가 데이터
!pip install finance-datareader

## **데이터 수집**

- 주식 데이터 수집

In [ ]:
import pandas as pd

In [ ]:
import FinanceDataReader as fdr

kospi = fdr.StockListing('KOSPI')
kospi.head()

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,72000,1,300,0.42,71700,72300,71400,17823512,1283667032959,429824343600000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,514000,1,6000,1.18,506000,521000,504000,515495,265186657000,120276000000000,234000000,STK
2,000660,KR7000660001,SK하이닉스,KOSPI,,122400,1,200,0.16,122900,124400,121600,3579662,440233706194,89107489476000,728002365,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,731000,3,0,0.00,732000,734000,727000,63153,46182047000,52028194000000,71174000,STK
4,005490,KR7005490008,POSCO홀딩스,KOSPI,,584000,1,31000,5.61,560000,597000,554000,1831875,1064306301874,49389598320000,84571230,STK


In [ ]:
top3 = kospi.head(3)[['Code','Name']]
top3

,Code,Name
0,005930,삼성전자
1,373220,LG에너지솔루션
2,000660,SK하이닉스


In [ ]:
stock_close = pd.DataFrame()
for i in top3.index:
    temp = fdr.DataReader(top3.loc[i,'Code'], start='2022-01-01', end='2022-12-31')[['Close']].rename({'Close':top3.loc[i,'Name']}, axis=1)
    stock_close = pd.concat([stock_close, temp], axis=1)

In [ ]:
stock_close

,삼성전자,LG에너지솔루션,SK하이닉스
Date,,,
2022-01-03,78600,NaN,128500
2022-01-04,78700,NaN,128500
2022-01-05,77400,NaN,125500
2022-01-06,76900,NaN,125000
2022-01-07,78300,NaN,127000
...,...,...,...
2022-12-23,58100,455500.0,77800
2022-12-26,57900,457500.0,77000
2022-12-27,58100,457000.0,77000


- 일자별 각 종목 뉴스 제목 수집

In [ ]:
from user_agent import generate_user_agent, generate_navigator
user_agent = generate_user_agent()
user_agent

'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.127 Safari/537.36'

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
from tqdm.notebook import tqdm

In [ ]:
date_list = list(stock_close.index.strftime('%Y.%m.%d'))
print(date_list)

['2022.01.03', '2022.01.04', '2022.01.05', '2022.01.06', '2022.01.07', '2022.01.10', '2022.01.11', '2022.01.12', '2022.01.13', '2022.01.14', '2022.01.17', '2022.01.18', '2022.01.19', '2022.01.20', '2022.01.21', '2022.01.24', '2022.01.25', '2022.01.26', '2022.01.27', '2022.01.28', '2022.02.03', '2022.02.04', '2022.02.07', '2022.02.08', '2022.02.09', '2022.02.10', '2022.02.11', '2022.02.14', '2022.02.15', '2022.02.16', '2022.02.17', '2022.02.18', '2022.02.21', '2022.02.22', '2022.02.23', '2022.02.24', '2022.02.25', '2022.02.28', '2022.03.02', '2022.03.03', '2022.03.04', '2022.03.07', '2022.03.08', '2022.03.10', '2022.03.11', '2022.03.14', '2022.03.15', '2022.03.16', '2022.03.17', '2022.03.18', '2022.03.21', '2022.03.22', '2022.03.23', '2022.03.24', '2022.03.25', '2022.03.28', '2022.03.29', '2022.03.30', '2022.03.31', '2022.04.01', '2022.04.04', '2022.04.05', '2022.04.06', '2022.04.07', '2022.04.08', '2022.04.11', '2022.04.12', '2022.04.13', '2022.04.14', '2022.04.15', '2022.04.18', '2022

In [ ]:
def news_crawling(keyword):
    data = pd.DataFrame()
    for dt in tqdm(date_list):
        dt1 = dt.replace('.','')

        url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={dt}&de={dt}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{dt1}to{dt1}&is_sug_officeid=0&office_category=0&service_area=1'

        user_agent = generate_user_agent()
        headers = {'User-Agent':user_agent}

        res = requests.get(url, headers=headers)
        time.sleep(random.random())
        soup = bs(res.text, 'html.parser')

        title = [i.text for i in soup.find_all('a', class_='news_tit')]

        if len(title) < 1 : #크롤링이 아무것도 되지 않았으면 루프를 종료, 시간 텀을 가진 뒤 다시 크롤링
            break

        temp = pd.DataFrame({'keyword':keyword, 'title':title, 'date_ymd':dt1})
        data = pd.concat([data, temp])

    return data

In [ ]:
keyword_list = ['삼성전자','LG에너지솔루션','SK하이닉스']

news_data = pd.DataFrame()
for k in keyword_list:
    news = news_crawling(k)
    news_data = pd.concat([news_data, news])

  0%|          | 0/246 [00:00<?, ?it/s]

  0%|          | 0/246 [00:00<?, ?it/s]

  0%|          | 0/246 [00:00<?, ?it/s]

In [ ]:
news_data.to_excel('/content/drive/MyDrive/강의자료_황수현_Python/Part3) 파이썬 데이터 분석 프로젝트/data/실습8: 직접 크롤링하기 - 뉴스기사 키워드 추출을 통한 주식 종목 이슈 분석/news.xlsx')

## **질문 만들기**
- 종목별 주가 추이는?
- 일자별 종목별 뉴스 키워드는?

## **데이터 전처리**

- 형태소 분석

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

In [ ]:
news_data = pd.read_excel('/content/drive/MyDrive/강의자료_황수현_Python/Part3) 파이썬 데이터 분석 프로젝트/data/실습8: 직접 크롤링하기 - 뉴스기사 키워드 추출을 통한 주식 종목 이슈 분석/news.xlsx')

In [ ]:
news_data['Noun'] = news_data['title'].apply(lambda x: [i[0] for i in mecab.pos(x) if i[1] in ("NNG","NNP") and len(i[0]) > 1])
news_data

,Unnamed: 0,keyword,title,date_ymd,Noun
0,0,삼성전자,"[CES 2022] 삼성전자, QD-OLED 대신 크기 키운 마이크로 LED 전면에",20220103,"[삼성전자, 대신, 크기, 마이크, 전면]"
1,1,삼성전자,삼성전자 2022년 시무식,20220103,"[삼성전자, 시무식]"
2,2,삼성전자,"삼성전자 한종희·경계현 ""최고의 '고객경험' 줘야…다시 한번 바꾸자""",20220103,"[삼성전자, 한종희, 경계현, 최고, 고객, 경험]"
3,3,삼성전자,"'꼰대 문화' 버려라, 실패해도 좋아…삼성전자 파격 신년사",20220103,"[꼰대, 문화, 실패, 삼성전자, 파격, 신년사]"
4,4,삼성전자,"[신년사]삼성전자 한종희·경계현 ""다시 한번 바꿔야 한다""",20220103,"[신년사, 삼성전자, 한종희, 경계현]"
...,...,...,...,...,...
7375,7375,SK하이닉스,"""연말 대목 효과 없었다""…SK하이닉스 이어 삼성까지 적자 예고",20221229,"[연말, 대목, 효과, 하이닉스, 삼성, 적자, 예고]"
7376,7376,SK하이닉스,SK하이닉스 이어 삼성도 적자 예고…극심한 반도체 한파,20221229,"[하이닉스, 삼성, 적자, 예고, 반도체, 한파]"
7377,7377,SK하이닉스,테슬라 ELS 하룻밤 새 1700억 증발 위기… 잠 못드는 투자자 [마켓워치],20221229,"[테슬라, 하룻밤, 증발, 위기, 투자자, 마켓, 워치]"
7378,7378,SK하이닉스,"[생생경제] 미중 사이에 낀 우리나라 경제, 내년 전망은?",20221229,"[경제, 사이, 우리나라, 경제, 내년, 전망]"


In [ ]:
news_data['date_ym'] = [str(i)[:6] for i in news_data['date_ymd']]
news_keyword = news_data.groupby(['keyword','date_ym'])[['Noun']].sum().reset_index()
news_keyword

,keyword,date_ym,Noun
0,LG에너지솔루션,202201,"[부회장, 권영수, 조직, 문화, 혁신안, 발표, 상장, 너지, 솔루션, 적정, 글..."
1,LG에너지솔루션,202202,"[배터리, 공장, 공모, 대비, 너지, 솔루션, 오늘, 거래, 하락, 시작, 전기차..."
2,LG에너지솔루션,202203,"[아프리카, 뉴딜, 지수, 편입, 화학, 제외, 기관, 매수, 회복, 급등, 탈환,..."
3,LG에너지솔루션,202204,"[세계, 격차, 미칼, 부회장, 신임, 대표, 김종현, 사장, 원통형, 배터리, 삼..."
4,LG에너지솔루션,202205,"[화재, 공정, 개선, 전극, 코팅, 이상, 현상, 발견, 호주, 라이온, 온과, ..."
5,LG에너지솔루션,202206,"[스텔, 란티스, 합작, 법인, 사명, 넥스트, 스타, 에너지, 글로벌, 대학생, ..."
6,LG에너지솔루션,202207,"[스탁, 너지, 솔루션, 넥스트, 큐어, 종합, 전지, 투자, 재검토, 보호, 예수..."
7,LG에너지솔루션,202208,"[수산, 인더스, 트리, 코스피, 상장, 올해, 유망, 배터리, 스타트업, 발굴, ..."
8,LG에너지솔루션,202209,"[합작, 현지, 공장, 전기, 배터리, 시제품, 생산, 시작, 임팩트, 투자, 요건..."
9,LG에너지솔루션,202210,"[구광모, 폴란드, 총리, 부산, 엑스포, 지지, 요청, 구광모, 대표, 폴란드, ..."


In [ ]:
from collections import Counter

In [ ]:
news_keyword['most_common'] = news_keyword['Noun'].apply(lambda x: dict(Counter(x).most_common()))
news_keyword

,keyword,date_ym,Noun,most_common
0,LG에너지솔루션,202201,"[부회장, 권영수, 조직, 문화, 혁신안, 발표, 상장, 너지, 솔루션, 적정, 글...","{'너지': 59, '솔루션': 59, '청약': 47, '상장': 35, '역대'..."
1,LG에너지솔루션,202202,"[배터리, 공장, 공모, 대비, 너지, 솔루션, 오늘, 거래, 하락, 시작, 전기차...","{'솔루션': 46, '너지': 44, '배터리': 16, '상장': 16, '특징..."
2,LG에너지솔루션,202203,"[아프리카, 뉴딜, 지수, 편입, 화학, 제외, 기관, 매수, 회복, 급등, 탈환,...","{'솔루션': 46, '너지': 42, '배터리': 35, '코스피': 29, '공..."
3,LG에너지솔루션,202204,"[세계, 격차, 미칼, 부회장, 신임, 대표, 김종현, 사장, 원통형, 배터리, 삼...","{'배터리': 41, '분기': 34, '삼성': 23, '코스피': 22, '너지..."
4,LG에너지솔루션,202205,"[화재, 공정, 개선, 전극, 코팅, 이상, 현상, 발견, 호주, 라이온, 온과, ...","{'배터리': 39, '솔루션': 23, '너지': 22, '투자': 19, '화학..."
5,LG에너지솔루션,202206,"[스텔, 란티스, 합작, 법인, 사명, 넥스트, 스타, 에너지, 글로벌, 대학생, ...","{'배터리': 54, '너지': 27, '솔루션': 27, '투자': 22, '코스..."
6,LG에너지솔루션,202207,"[스탁, 너지, 솔루션, 넥스트, 큐어, 종합, 전지, 투자, 재검토, 보호, 예수...","{'배터리': 50, '솔루션': 27, '너지': 26, '화학': 24, '실적..."
7,LG에너지솔루션,202208,"[수산, 인더스, 트리, 코스피, 상장, 올해, 유망, 배터리, 스타트업, 발굴, ...","{'배터리': 64, '코스피': 25, '솔루션': 20, '너지': 19, '공..."
8,LG에너지솔루션,202209,"[합작, 현지, 공장, 전기, 배터리, 시제품, 생산, 시작, 임팩트, 투자, 요건...","{'배터리': 42, '솔루션': 32, '너지': 25, '코스피': 14, '구..."
9,LG에너지솔루션,202210,"[구광모, 폴란드, 총리, 부산, 엑스포, 지지, 요청, 구광모, 대표, 폴란드, ...","{'분기': 33, '배터리': 31, '솔루션': 26, '실적': 26, '너지..."


In [ ]:
def noun_clean(keyword, noun):
    if keyword == 'LG에너지솔루션':
        noun_clean = [i for i in noun if i not in ['너지','솔루션']]
    elif keyword == 'SK하이닉스':
        noun_clean = [i for i in noun if i not in ['하이닉스']]
    else:
        noun_clean = [i for i in noun if i not in ['삼성전자','삼성','전자']]

    return noun_clean

In [ ]:
news_keyword['noun_clean'] = news_keyword.apply(lambda x: noun_clean(x['keyword'], x['Noun']), axis=1)
news_keyword['most_common'] = news_keyword['noun_clean'].apply(lambda x: dict(Counter(x).most_common()))
news_keyword

,keyword,date_ym,Noun,most_common,noun_clean
0,LG에너지솔루션,202201,"[부회장, 권영수, 조직, 문화, 혁신안, 발표, 상장, 너지, 솔루션, 적정, 글...","{'청약': 47, '상장': 35, '역대': 20, '배터리': 15, '최대'...","[부회장, 권영수, 조직, 문화, 혁신안, 발표, 상장, 적정, 글로벌, 전기, 승..."
1,LG에너지솔루션,202202,"[배터리, 공장, 공모, 대비, 너지, 솔루션, 오늘, 거래, 하락, 시작, 전기차...","{'배터리': 16, '상장': 16, '특징': 15, '코스피': 13, '화학...","[배터리, 공장, 공모, 대비, 오늘, 거래, 하락, 시작, 전기차, 대분, 수산화..."
2,LG에너지솔루션,202203,"[아프리카, 뉴딜, 지수, 편입, 화학, 제외, 기관, 매수, 회복, 급등, 탈환,...","{'배터리': 35, '코스피': 29, '공매도': 19, '화학': 18, '신...","[아프리카, 뉴딜, 지수, 편입, 화학, 제외, 기관, 매수, 회복, 급등, 탈환,..."
3,LG에너지솔루션,202204,"[세계, 격차, 미칼, 부회장, 신임, 대표, 김종현, 사장, 원통형, 배터리, 삼...","{'배터리': 41, '분기': 34, '삼성': 23, '코스피': 22, '실적...","[세계, 격차, 미칼, 부회장, 신임, 대표, 김종현, 사장, 원통형, 배터리, 삼..."
4,LG에너지솔루션,202205,"[화재, 공정, 개선, 전극, 코팅, 이상, 현상, 발견, 호주, 라이온, 온과, ...","{'배터리': 39, '투자': 19, '화학': 17, '삼성': 15, '코스피...","[화재, 공정, 개선, 전극, 코팅, 이상, 현상, 발견, 호주, 라이온, 온과, ..."
5,LG에너지솔루션,202206,"[스텔, 란티스, 합작, 법인, 사명, 넥스트, 스타, 에너지, 글로벌, 대학생, ...","{'배터리': 54, '투자': 22, '코스피': 21, '전기차': 16, '삼...","[스텔, 란티스, 합작, 법인, 사명, 넥스트, 스타, 에너지, 글로벌, 대학생, ..."
6,LG에너지솔루션,202207,"[스탁, 너지, 솔루션, 넥스트, 큐어, 종합, 전지, 투자, 재검토, 보호, 예수...","{'배터리': 50, '화학': 24, '실적': 20, '분기': 19, '코스피...","[스탁, 넥스트, 큐어, 종합, 전지, 투자, 재검토, 보호, 예수, 해제, 신저,..."
7,LG에너지솔루션,202208,"[수산, 인더스, 트리, 코스피, 상장, 올해, 유망, 배터리, 스타트업, 발굴, ...","{'배터리': 64, '코스피': 25, '공장': 17, '기업': 15, '전기...","[수산, 인더스, 트리, 코스피, 상장, 올해, 유망, 배터리, 스타트업, 발굴, ..."
8,LG에너지솔루션,202209,"[합작, 현지, 공장, 전기, 배터리, 시제품, 생산, 시작, 임팩트, 투자, 요건...","{'배터리': 42, '코스피': 14, '구광모': 14, '화학': 13, '기...","[합작, 현지, 공장, 전기, 배터리, 시제품, 생산, 시작, 임팩트, 투자, 요건..."
9,LG에너지솔루션,202210,"[구광모, 폴란드, 총리, 부산, 엑스포, 지지, 요청, 구광모, 대표, 폴란드, ...","{'분기': 33, '배터리': 31, '실적': 26, '화학': 18, '삼성'...","[구광모, 폴란드, 총리, 부산, 엑스포, 지지, 요청, 구광모, 대표, 폴란드, ..."


## **분석**

In [ ]:
import plotly.express as px

for y in ['삼성전자','LG에너지솔루션','SK하이닉스']:
    fig = px.line(data_frame = stock_close.reset_index(), x='Date', y=y)
    fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothic')

from wordcloud import WordCloud

path = '/user/share/fonts/truetype/nanum/NanumGothic.ttf'
wc = WordCloud(font_path = path,
               background_color='white',
               width=1000,
               height=1000)

date_ym_list = sorted(news_keyword['date_ym'].unique())

for k in ['삼성전자','LG에너지솔루션','SK하이닉스']:
    fig = plt.figure(figsize=(25,20))
    for i in range(12):
        wc.generate_from_frequencies(news_keyword.query(f'keyword == "{k}" and date_ym == "{date_ym_list[i]}"')['most_common'].values[0])
        ax = fig.add_subplot(3,4,i+1)
        ax.imshow(wc, interpolation='bilinear')
        ax.set_xlabel(date_ym_list[i])
        ax.set_xticks([]), ax.set_yticks([])
    fig.suptitle(k)
    fig.tight_layout()
    plt.show()

In [ ]:
stock_close['date_ym'] = [i[:6] for i in stock_close.index.strftime('%Y%m%d')]
stock_close.head()

,삼성전자,LG에너지솔루션,SK하이닉스,date_ym
Date,,,,
2022-01-03,78600,NaN,128500,202201
2022-01-04,78700,NaN,128500,202201
2022-01-05,77400,NaN,125500,202201
2022-01-06,76900,NaN,125000,202201
2022-01-07,78300,NaN,127000,202201


In [ ]:
news_keyword['TOP20'] = news_keyword['noun_clean'].apply(lambda x: str(dict(Counter(x).most_common(20))))

stock_close_keyword = stock_close.reset_index()\
.merge(news_keyword.query('keyword == "삼성전자"')[['date_ym','TOP20']], on='date_ym').rename({'TOP20':'삼성전자 월별 TOP20 키워드'}, axis=1)\
.merge(news_keyword.query('keyword == "LG에너지솔루션"')[['date_ym','TOP20']], on='date_ym').rename({'TOP20':'LG에너지솔루션 월별 TOP20 키워드'}, axis=1)\
.merge(news_keyword.query('keyword == "SK하이닉스"')[['date_ym','TOP20']], on='date_ym').rename({'TOP20':'SK하이닉스 월별 TOP20 키워드'}, axis=1)
stock_close_keyword

,Date,삼성전자,LG에너지솔루션,SK하이닉스,date_ym,삼성전자 월별 TOP20 키워드,LG에너지솔루션 월별 TOP20 키워드,SK하이닉스 월별 TOP20 키워드
0,2022-01-03,78600,NaN,128500,202201,"{'반도체': 24, '매출': 23, '역대': 15, '작년': 13, '최대'...","{'청약': 47, '상장': 35, '역대': 20, '배터리': 15, '최대'...","{'반도체': 30, '코스피': 23, '삼성전자': 22, '기업': 14, '..."
1,2022-01-04,78700,NaN,128500,202201,"{'반도체': 24, '매출': 23, '역대': 15, '작년': 13, '최대'...","{'청약': 47, '상장': 35, '역대': 20, '배터리': 15, '최대'...","{'반도체': 30, '코스피': 23, '삼성전자': 22, '기업': 14, '..."
2,2022-01-05,77400,NaN,125500,202201,"{'반도체': 24, '매출': 23, '역대': 15, '작년': 13, '최대'...","{'청약': 47, '상장': 35, '역대': 20, '배터리': 15, '최대'...","{'반도체': 30, '코스피': 23, '삼성전자': 22, '기업': 14, '..."
3,2022-01-06,76900,NaN,125000,202201,"{'반도체': 24, '매출': 23, '역대': 15, '작년': 13, '최대'...","{'청약': 47, '상장': 35, '역대': 20, '배터리': 15, '최대'...","{'반도체': 30, '코스피': 23, '삼성전자': 22, '기업': 14, '..."
4,2022-01-07,78300,NaN,127000,202201,"{'반도체': 24, '매출': 23, '역대': 15, '작년': 13, '최대'...","{'청약': 47, '상장': 35, '역대': 20, '배터리': 15, '최대'...","{'반도체': 30, '코스피': 23, '삼성전자': 22, '기업': 14, '..."
...,...,...,...,...,...,...,...,...
241,2022-12-23,58100,455500.0,77800,202212,"{'반도체': 23, '사장': 17, '이재용': 13, '내년': 13, '회장...","{'배터리': 44, '투자': 23, '코스피': 16, '내년': 14, '테슬...","{'반도체': 49, '삼성전자': 42, '신저': 25, '전망': 20, '적..."
242,2022-12-26,57900,457500.0,77000,202212,"{'반도체': 23, '사장': 17, '이재용': 13, '내년': 13, '회장...","{'배터리': 44, '투자': 23, '코스피': 16, '내년': 14, '테슬...","{'반도체': 49, '삼성전자': 42, '신저': 25, '전망': 20, '적..."
243,2022-12-27,58100,457000.0,77000,202212,"{'반도체': 23, '사장': 17, '이재용': 13, '내년': 13, '회장...","{'배터리': 44, '투자': 23, '코스피': 16, '내년': 14, '테슬...","{'반도체': 49, '삼성전자': 42, '신저': 25, '전망': 20, '적..."
244,2022-12-28,56600,436500.0,76000,202212,"{'반도체': 23, '사장': 17, '이재용': 13, '내년': 13, '회장...","{'배터리': 44, '투자': 23, '코스피': 16, '내년': 14, '테슬...","{'반도체': 49, '삼성전자': 42, '신저': 25, '전망': 20, '적..."


In [ ]:
for y in ['삼성전자','LG에너지솔루션','SK하이닉스']:
    fig = px.line(data_frame = stock_close_keyword, x='Date', y=y, hover_data=f'{y} 월별 TOP20 키워드')
    fig.show()